In [12]:
# !pip install pydicom
# !pip install dicomweb_client
# !pip install tqdm

In [2]:
from dicomweb_client.api import DICOMwebClient
import pydicom
import os
import time
import datetime
from tqdm import tqdm
import csv
import random

url = 'http://localhost:8080/dcm4chee-arc/aets/DCM4CHEE/rs'

client = DICOMwebClient(url)

Funkcja obliczania długości odczytywania plików z bazy:

In [11]:
sample_size = 100
def get_average_read_time(patient_UID):
    # warmup reads (for the first time search time is larger for some reason - idk why)
    client.search_for_studies(search_filters={'PatientID': str(patient_UID)})
    client.search_for_studies(search_filters={'PatientID': str(patient_UID)})
    
    samples = []
    for i in range(sample_size):
        samples.append(random.randint(0, patient_UID))
    
    start_time = time.time()
    
    for sample in samples:
        print(sample)
        client.search_for_studies(search_filters={'PatientID': str(sample)})
    
    search_time = (time.time() - start_time) / sample_size
    
    print("Patient UDI: " + str(patient_UID) + "; average search time: " + str(search_time) )
    return search_time

get_average_read_time(100000)

empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response


97482
92283
12959
51146
39929
13594
97541
64682
83353
58944
97052
2025
45250
23153
78806
26263
40757
87140
60591
23807
88454


empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response


90955
78345
66194
43095
66121
33067
43654
51146
52143
63342
57770
81993
85142
16706
25039
64355
84686
27591
15440
52032
97942


empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response


...

empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response


60823
41310
31687
83533
85484
21074
65468
12104
93078
8522
22238
951
95609
57326
6211
25555
91953
95191
74961
1039
54833
3798


empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response
empty response


94426
66821
44230
83400
42433
9996
59059
16786
85915
12821
97196
90025
69344
10011
92084
49752
36704
Patient UDI: 100000; average search time: 0.00931218385696411


0.00931218385696411

Przygotowanie pliku z logami:

In [3]:
path = "log/uploading_dicom_to_pacs_logs-" + str(time.strftime("%Y%m%d-%H%M%S")) + ".csv"
print("logs path: " + path)
header = ['timestamp', 'number_of_images', 'last_1k_dicoms_uploadtime', 'experiment_time', 'pacs_read_time']

with open(path, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

logs path: log/uploading_dicom_to_pacs_logs-20220524-225458.csv


Funkcja wgrywania paczki dicomów do pacsa:

In [4]:
def upload_1k_dicoms_into_pacs(images_prefix):
    start_time = time.time()
    
    for i in range(1000):
        uid = str(images_prefix) + str(i)
        
        dicom.SeriesInstanceUID = uid
        dicom.StudyInstanceUID = uid
        dicom.FrameOfReferenceUID = uid
        dicom.SOPInstanceUID = uid
        dicom.PatientID = uid
        client.store_instances(datasets=[dicom])
    
    return( time.time() - start_time )   

### Zapełnianie bazy dicomami:

Przygotowanie dicoma:

In [5]:
dicom = pydicom.dcmread('test_images/IM-0014-0001.dcm')
dicom.PixelData = bytes()

Pętla wgrywania:

In [ ]:
uid_prefix = dicom.SeriesInstanceUID + '.'

experiment_start_time = time.time()

for i in tqdm(range(250,1000)):
    upload_time = upload_1k_dicoms_into_pacs(uid_prefix + str(i))
    
#     timestamp = datetime.datetime.now()
#     number_of_images = i*1000
#     last_1k_dicoms_uploadtime = upload_time
#     experiment_time = time.time() - experiment_start_time
#     pacs_read_time = get_average_read_time(number_of_images)
    
#     data = [str(timestamp), str(number_of_images), str(last_1k_dicoms_uploadtime), str(experiment_time), str(pacs_read_time)]
    
#     with open(path, 'a', encoding='UTF8', newline='') as f:
#         csv.writer(f).writerow(data)
    

 84%|███████████████████████████████████████████████████████████████▋            | 629/750 [9:11:46<1:46:21, 52.74s/it]